In [7]:
a = rand(10);
b = Dict("a" => rand(10));

In [8]:
function serializewrite( fname :: String 
                       , data )
    fname = split(fname, ".jls")[1]
    fname = split(fname, "__jversion_")[1]
    jls_file  = open( string( fname
                            , "__jversion_"
                            , replace( string(VERSION)
                                     , "."
                                     , "_"
                                     )
                            , ".jls"
                            )
                    , "w"
                    )
    serialize( jls_file
             , data
             )
    close(jls_file)
end;
#---------------------------------------------------------------------------------------------------
function save_kernel(filename)
    namespace = names(Main)
    # masking out the modules
    mask = typeof.(eval.(namespace)) .!= Module
    # masking out the user defined functions (these don't need to be saved)
    mask .*= map(x -> x[1], string.(typeof.(eval.(namespace)))) .!= '#'
    
    outdict = Dict()
    for name in namespace[mask]
        outdict[name] = eval(name)
    end
    serializewrite(filename, outdict)
end;

In [9]:
save_kernel("boo")

At this point you can restart the kernel and then run the following

In [1]:
function serializeread( fname :: String )

    if !( string( replace( string(VERSION)
                         , "."
                         , "_"
                         )
                , ".jls"
                )
          ==
          split(fname, "__jversion_")[2]
        )
        error( string( "Attempting to read serialized file generated by a different version of "
                     , "Julia . . . you will need to remake the file =("
                     )
             )
    end

    return deserialize(open(fname, "r"))
end;
#---------------------------------------------------------------------------------------------------
function load_kernel(filename)
    indict = serializeread(filename)
    for (k,v) in zip(keys(indict),values(indict))
        exp = :($k = $v)
        eval(exp)
    end
end;

In [2]:
load_kernel("boo__jversion_0_6_2.jls")

In [3]:
a

10-element Array{Float64,1}:
 0.0743878
 0.912397 
 0.578943 
 0.131044 
 0.953834 
 0.766621 
 0.0716108
 0.589877 
 0.261829 
 0.481318 

In [4]:
b

Dict{String,Array{Float64,1}} with 1 entry:
  "a" => [0.774685, 0.134833, 0.844015, 0.409108, 0.939711, 0.0354891, 0.598207…